In [1]:
# Install pyspark and findspark
!pip install --ignore-install -q pyspark
# Install findspark library
!pip install --ignore-install -q findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 16.5 MB/s eta 0:00:00


In [2]:
# Import findspark
import findspark
findspark.init()

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
# Titanic logistic regression
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Titanic logistic regression').getOrCreate()
# Load dataset
data = spark.read.csv('/content/drive/MyDrive/titanic.csv',inferSchema=True,header=True)

data.printSchema()
data.columns

# Use numeric columns
my_cols = data.select(['Survived',
'Pclass',
'Sex',
'Age',
'SibSp',
'Parch',
'Fare',
'Embarked'])
# drop missing data
my_final_data = my_cols.na.drop()
# working with categorical columns
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)
# Assign number to each of category, e.g: male female -> 0 1
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
# One hot encoder: male ->[1, 0], female -> [0,1], each category will be converted into vector
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')
# assemble all into features
assembler = VectorAssembler(inputCols=['Pclass',
'SexVec',
'Age',
'SibSp',
'Parch',
'Fare',
'EmbarkVec'], outputCol='features')
from pyspark.ml.classification import LogisticRegression
# Pipeline, create stages for different steps
from pyspark.ml import Pipeline
log_reg_titantic = LogisticRegression(featuresCol='features', labelCol='Survived')
# create pipeline
pipeline = Pipeline(stages=[gender_indexer,
                    embark_indexer,
                    gender_encoder,
                    embark_encoder,
                    assembler,
                    log_reg_titantic])
train_titanic_data, test_titanic_data = my_final_data.randomSplit([0.7,0.3])
# fit model
fit_model = pipeline.fit(train_titanic_data)
# test model using test data
results = fit_model.transform(test_titanic_data)
# evaluate model
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')
results.select('Survived','prediction').show()
AUC = my_eval.evaluate(results)
print("AUC is: "+ str(AUC))

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only show

In [7]:
spark.stop()